In [ ]:
import os
import random
import torch

from datasets import load_dataset
from transformers import AutoTokenizer
import stanza

# filter the length < 7
def clean_data(example):
    tmp = re.sub(r'^(\d+).', '', example)
    tmp = re.sub(r'^\((ix|iv|v?i{0,3})\)', '', tmp)
    tmp = re.sub(r'^\((\d+)\)', '', tmp)
    new_str = tmp.translate(str.maketrans('','',string.punctuation)).replace("’s",'').replace("s’",'s')
    return new_str

# date prepared for training
def tokenize_function(examples):
    lower = [clean_data(x.lower()) for x in examples["paragraphs"]]
    lower = filter(lambda x : len(x) >7, lower )
    result = tokenizer(lower)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# split the paragraphs to sentence
# [(sent1,sent2,isnext)]
# def paragraphs_split(example):
#     nlp = stanza.Pipeline(lang='en', processors='tokenize')
#     doc = nlp(example["paragraphs"])
#     sentences = [sentence.text for sentence in doc.sentences]
#     _get_nsp_data_from_paragraph


def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # `paragraphs` is a list of lists of lists
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next


def _get_nsp_data_from_paragraph(paragraph, paragraphs, max_len=512):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # Consider 1 '<cls>' token and 2 '<sep>' tokens
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

hklii_dataset_ch = load_dataset("json", data_files="/home/xijia/nlp/data_prepare/data/HKLII_zh.json")
hklii_dataset_ch = hklii_dataset_ch["train"]
hklii_dataset_en = load_dataset("json", data_files="/home/huijie/legal/huggface/data_prepare/HKLII-online/HKLII.json")
hklii_dataset_en = hklii_dataset_en["train"]



hklii = train paragraphs sentences

1. shuffle paragraphs
2. get the setences in paragraph
2. for sentences in paragraphs:
    for sentence in sentences



In [ ]:
def paragraphs_split(example):
    nlp = stanza.Pipeline(lang='en', processors='tokenize')
    doc = nlp(example["paragraphs"])
    sentences = [sentence.text for sentence in doc.sentences]
    example["sentences"] = sentences
    return example
sampled = hklii_dataset_en[:10]
sampled.map(paragraphs_split,nproc =4)